In [24]:
region_df = spark.read.load('abfss://khd-datalake@mgrhdsantanderdl2.dfs.core.windows.net/tpc-h/region.csv', format='csv', header=True, sep='|').drop('empty')
region_df.show()

+-----------+-----------+--------------------+
|R_REGIONKEY|     R_NAME|           R_COMMENT|
+-----------+-----------+--------------------+
|          0|     AFRICA|lar deposits. bli...|
|          1|    AMERICA|hs use ironic, ev...|
|          2|       ASIA|ges. thinly even ...|
|          3|     EUROPE|ly final courts c...|
|          4|MIDDLE EAST|uickly special ac...|
+-----------+-----------+--------------------+

In [11]:
%%spark

import com.microsoft.spark.sqlanalytics.utils.Constants
import org.apache.spark.sql.SqlAnalyticsConnector._
val df = spark.read.sqlanalytics("stdmgrkhd.dbo.nation") 
df.write.mode("overwrite").saveAsTable("tempTable_Nation")



import com.microsoft.spark.sqlanalytics.utils.Constants
import org.apache.spark.sql.SqlAnalyticsConnector._
StructuredStream-spark package version: 2.4.5-1.3.1
df: org.apache.spark.sql.DataFrame = [N_NATIONKEY: int, N_NAME: string ... 2 more fields]
StructuredStream-spark package version: 2.4.5-1.3.1
StructuredStream-spark package version: 2.4.5-1.3.1

In [35]:
nations_df = spark.sql("select * from tempTable_Nation")
result_df = region_df.join(nations_df, region_df.R_REGIONKEY == nations_df.N_REGIONKEY) \
    .select(nations_df.N_NAME, region_df.R_NAME) \
    .withColumnRenamed("N_NAME", "nation_name") \
    .withColumnRenamed("R_NAME", "region_name") \
    .orderBy("region_name")
display(result_df)

result_df.write.parquet("abfss://khd-datalake@mgrhdsantanderdl2.dfs.core.windows.net/tpc-h/processed/natregion.parquet", partitionBy="region_name")

+-----------+--------------------+-----------+--------------------+
|N_NATIONKEY|              N_NAME|N_REGIONKEY|            NCOMMENT|
+-----------+--------------------+-----------+--------------------+
|          1|ARGENTINA        ...|          1|al foxes promise ...|
|          8|INDIA            ...|          2|ss excuses cajole...|
|         15|MOROCCO          ...|          0|rns. blithely bol...|
|         22|RUSSIA           ...|          3| requests against...|
|          6|FRANCE           ...|          3|refully final req...|
|         13|JORDAN           ...|          4|ic deposits are b...|
|         20|SAUDI ARABIA     ...|          4|ts. silent reques...|
|          2|BRAZIL           ...|          1|y alongside of th...|
|          9|INDONESIA        ...|          2| slyly express as...|
|         16|MOZAMBIQUE       ...|          0|s. ironic, unusua...|
|         23|UNITED KINGDOM   ...|          3|eans boost carefu...|
|          5|ETHIOPIA         ...|          0|ve